# Fixstars Amplify Annealing Engine を用いた解法

Amplify Annealing Engine (AE) はUBQP(=QUBO, イジングモデル)形式の問題に対するシミュレーテッドアニーリング(SA)をGPU上で効率的に実装したマシン．

使用する場合は以下のページからユーザ登録(試すだけなら無料)をしてトークンを発行する．

https://amplify.fixstars.com/ja/

In [1]:
import numpy as np
from util import read_qap
from amplify import VariableGenerator, Poly
from amplify import one_hot, einsum
from amplify import FixstarsClient, solve

folder = '../data/qap/problem/'

with open('token.txt', 'r') as f:
    token = f.read()

In [2]:
n, f, d = read_qap(folder +"tai20a.dat")

In [3]:
gen = VariableGenerator()
q = gen.array("Binary", shape=(n, n))

# matrix を用いた目的関数の構築 (d, f が辞書なので今回は使用できない)
#matrix = gen.matrix("Binary", n, n)
#q = matrix.variable_array
# Objective function (set to matrix.quadratic)
#np.einsum("ij,kl->ikjl", d, f, out=matrix.quadratic)

objective = 0
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                objective += q[i,k] * q[j,l] * d[i,j] * f[k,l]

# einsum による高速な目的関数の構築 (d, f が辞書なので今回は使用できない)
#objective: Poly = einsum('ij,kl,ik,jl->', d, f, q, q)


# Constraint Conditions
col_constraint = one_hot(q, axis=0)
row_constraint = one_hot(q, axis=1)
constraint = col_constraint + row_constraint

In [4]:
penalty_weight = max(d.values()) * max(f.values()) * (n - 1)

model = objective + penalty_weight * constraint

In [8]:
client = FixstarsClient()
client.token = token

import datetime
client.parameters.timeout = datetime.timedelta(seconds=1)

In [9]:
result = solve(model, client)

result.best.objective

718716.0

In [10]:
q_values = q.evaluate(result.best.values)

print(q_values)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.